In [7]:
import pandas as pd
import hashlib
import re
from cryptography.fernet import Fernet

# Загрузка данных
file_path = '/content/MyDrive/Laptop_price.csv' # Замените на путь к файлу
try:
    df = pd.read_csv(file_path)  # Читаем CSV-файл и загружаем его в DataFrame
except Exception as e:
    print(f"Ошибка загрузки файла: {e}")




# 1. Проверка CSV на уязвимости (CSV Injection)
# Опасные символы могут использоваться для атак в Excel и других программах
# Функция проверяет, начинаются ли значения в текстовых столбцах с этих символов
def check_csv_injection(df):
    dangerous_chars = ('=', '+', '-', '@')
    for col in df.select_dtypes(include=['object']).columns:
        if df[col].astype(str).apply(lambda x: x.startswith(dangerous_chars)).any():
            print(f"Обнаружены потенциальные CSV-инъекции в столбце {col}!")
        else:
            print(f"Столбец {col} безопасен.")
    check_csv_injection(df)

# 2. Фильтрация данных от SQL-инъекций и XSS-атак
# SQL-инъекции используются для взлома баз данных, а XSS – для атак на пользователей
# Функция заменяет опасные конструкции на строку '[BLOCKED]'
def clean_input(value):
    sql_keywords = ["SELECT", "DROP", "DELETE", "INSERT", "UPDATE", "ALTER", "UNION"]
    xss_patterns = [r'<script.*?>.*?</script>', r'javascript:.*', r'onerror=.*']
    for keyword in sql_keywords:
        if keyword.lower() in value.lower():
            return "[BLOCKED]"
    for pattern in xss_patterns:
        if re.search(pattern, value, re.IGNORECASE):
            return "[BLOCKED]"
    return value
df = df.applymap(lambda x: clean_input(str(x)) if isinstance(x, str) else x)
print ("Фильтрация данных завершена.")

# 3. Хеширование столбца с ценами (SHA-256)
# Хеширование используется для защиты конфиденциальных данных, таких как пароли
# В данном случае мы хешируем цену ноутбука, чтобы скрыть оригинальное значение
def hash_price(price):
    return hashlib.sha256(str(price).encode()).hexdigest()

df['Price_Hashed'] = df['Price'].apply(hash_price)
print("Столбец с хешированными ценами добавлен.")

# 4. Шифрование данных (например, цены ноутбука)
# Шифрование позволяет скрыть данные, но при этом их можно расшифровать при наличии ключа
# Генерируем ключ и шифруем цену ноутбука
key = Fernet.generate_key()
cipher = Fernet(key)

def encrypt_price(price):
    return cipher.encrypt(str(price).encode()).decode()

def decrypt_price(encrypted_price):
    return cipher.decrypt(encrypted_price.encode()).decode()

df['Price_Encrypted'] = df['Price'].apply(encrypt_price)
print("Столбец с зашифрованными ценами добавлен.")


# Сохранение обработанных данных
output_path = "Laptop_price_secure.csv"
df.to_csv(output_path, index=False)
print(f"Обработанный файл сохранен: {output_path}")


Ошибка загрузки файла: [Errno 2] No such file or directory: '/content/MyDrive/Laptop_price.csv'


NameError: name 'df' is not defined